<a href="https://colab.research.google.com/github/adrien-simard/adrien-simard-football-dataset/blob/master/EnglishPlayers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [0]:
p1=pd.read_csv("player1.csv", sep=',', encoding="latin1")


In [0]:
p2=pd.read_csv("player2.csv", sep=',', encoding="latin1")

In [0]:
p3=pd.read_csv("players3.csv", sep=',', encoding="latin1")

In [0]:
p4=pd.read_csv("players4.csv", sep=',', encoding="latin1")

In [0]:
p5=pd.read_csv("players5.csv", sep=',', encoding="latin1")

In [0]:
tot = pd.concat([p1, p2,p3,p4,p5],sort=False)
df =tot[tot.columns[1:21]]
df.columns = ['Player','Nation','Pos','Squad', 'Age','Born','MP',	'Starts',	'Min',	'Goals',	'Assists',	'PenalKick','PKattempt',	'CardY',	'CardR',	'Goals/90',	'Ast/90',	'G+A/90',	'G-PK/90',	'G+A-PK']


In [0]:
df.to_csv("EnglishPlayer.csv")